In [5]:
import os
import open3d as o3d
import numpy as np
import pickle
import pymeshlab
from shutil import copyfile

import random

In [6]:
ycb_dir = "ycb_different_grasps"

# Path to the directory with all the folders with the pickle pressure files
pressure_path = "C:/Users/lucas/Desktop/UPC/MIT/manopth/outputs/graspit_to_mano/ycb/"

In [8]:
test = [
        "036_wood_block",
        "035_power_drill",
        "026_sponge",
        "044_flat_screwdriver",
        "043_phillips_screwdriver",
        "058_golf_ball",
        "065-i_cups",
        "003_cracker_box",
        "022_windex_bottle",
        "029_plate",
        "013_apple",
        "033_spatula",
        "052_extra_large_clamp",
        "005_tomato_soup_can",
        "056_tennis_ball",
        "070-a_colored_wood_blocks",
        "073-d_lego_duplo"
]

# Convert pressure info to pointcloud

In [10]:
pressure_pointcloud_path = os.path.join(ycb_dir, "5_pressure_pointcloud")

if not os.path.exists(pressure_pointcloud_path):
    os.makedirs(pressure_pointcloud_path)

In [21]:
n_samples = [5, 10, 20, 50]

In [28]:
get_obj_name = lambda name : name[:len(name) - 14]
obj_names = []

for n in n_samples:
    for obj_test in test:
        obj_folder = obj_test +"_pressure_info"

        pkls_path = os.path.join(pressure_path, obj_folder)
        obj_name = get_obj_name(obj_folder)

        # Get sensor info from every pkl file
        all_sensors_xyz = []
        all_sensors_pressure = []
        
        try:
            for pkl_name in random.sample(os.listdir(pkls_path), n):
                pkl_path = os.path.join(pkls_path, pkl_name)


                pressure_info = pickle.load( open( pkl_path, "rb") )

                # Load the sensors pressure and take only the ones that have pressure greater than 0
                sensors_pressure = pressure_info['sensors_pressure']
                idx_sensors = np.nonzero(sensors_pressure)

                sensors_xyz = pressure_info['sensors_xyz'][idx_sensors]
                sensors_pressure = sensors_pressure[idx_sensors]

                all_sensors_xyz.append(sensors_xyz)
                all_sensors_pressure.append(sensors_pressure)

            sensors_xyz = np.concatenate(all_sensors_xyz)
            sensors_pressure = np.concatenate(all_sensors_pressure)

            # Point cloud of the geneated object
            pcd_gen = o3d.geometry.PointCloud()
            pcd_gen.points = o3d.utility.Vector3dVector(sensors_xyz)

            pcd_gen.estimate_normals()
            pcd_gen.orient_normals_consistent_tangent_plane(50)

            ply_pth = os.path.join(pressure_pointcloud_path, f"{obj_name}_{n}.ply")
            obj_names.append(f"{obj_name}_{n}")
            o3d.io.write_point_cloud(ply_pth, pcd_gen)
        except:
            pass

In [43]:


s = obj_names[3]


20

In [42]:
obj_names[3][:20]

'044_flat_screwdriver'

# Generate data for Convolutional Occupancy Networks
Generate the pointcloud.npz files

In [44]:
pressure_points_path = os.path.join(ycb_dir, "6_pressure_pointcloud")

if not os.path.exists(pressure_points_path):
    os.makedirs(pressure_points_path)
    
for obj_name in obj_names:
    s = obj_name
    pos = [pos for pos, char in enumerate(s) if char == '_'][-1]
    pc_pth = os.path.join(ycb_dir, "4_pointcloud", f"{obj_name[:pos]}.npz")
    pc_npz = np.load(pc_pth)

    translation = pc_npz['loc'].tolist()
    scale = pc_npz['scale'].item()

    ply_pth = os.path.join(pressure_pointcloud_path, f"{obj_name}.ply")
    pcd_pressure = o3d.io.read_point_cloud(ply_pth)

    points = (np.asarray(pcd_pressure.points) - translation) / scale
    normals = np.asarray(pcd_pressure.normals)

    npz_pth = os.path.join(pressure_points_path, obj_name)
    np.savez(npz_pth, 
             points=points, 
             normals=normals,
             loc=pc_npz['loc'],
             scale=pc_npz['scale']
             )

# Create dataset

In [45]:
dataset_pth = os.path.join(ycb_dir, "ycb_con")

if not os.path.exists(dataset_pth):
    os.makedirs(dataset_pth)

In [48]:
for obj_name in obj_names:
    s = obj_name
    pos = [pos for pos, char in enumerate(s) if char == '_'][-1]
    obj_dir = os.path.join(dataset_pth, obj_name)
    
    if not os.path.exists(obj_dir):
        os.makedirs(obj_dir)
        
    src_pth_ps = os.path.join(ycb_dir, "4_points", f"{obj_name[:pos]}.npz")
    dst_pth_ps = os.path.join(obj_dir, "points.npz")
    copyfile(src_pth_ps, dst_pth_ps)
    
    src_pth_pcd = os.path.join(pressure_points_path, f"{obj_name}.npz")
    dst_pth_pcd = os.path.join(obj_dir, "pointcloud.npz")
    copyfile(src_pth_pcd, dst_pth_pcd)

In [62]:
test_pth = os.path.join(dataset_pth, 'test.lst')

with open(test_pth, 'w') as filehandle:
    for obj_name in obj_names:
        filehandle.write('%s\n' % obj_name)

# Compute average number of points per grasp

In [18]:
get_obj_name = lambda name : name[:len(name) - 14]
obj_names = []

avg = []

for obj_folder in os.listdir(pressure_path):

    pkls_path = os.path.join(pressure_path, obj_folder)
    obj_name = get_obj_name(obj_folder)
    obj_names.append(obj_name)
    
    # Get sensor info from every pkl file
    all_sensors_xyz = []
    all_sensors_pressure = []

    for pkl_name in os.listdir(pkls_path):
        pkl_path = os.path.join(pkls_path, pkl_name)


        pressure_info = pickle.load( open( pkl_path, "rb") )

        # Load the sensors pressure and take only the ones that have pressure greater than 0
        sensors_pressure = pressure_info['sensors_pressure']
        idx_sensors = np.nonzero(sensors_pressure)
        
        avg.append(len(idx_sensors[0]))

In [19]:
sum(avg) / len(avg)

38.73090498376394

# To create dataset with different number of grasps

In [ ]:
get_obj_name = lambda name : name[:len(name) - 14]
obj_names = []
n_samples_dict = defaultdict(list)

for n in n_samples:
    for obj_folder in os.listdir(pressure_path):
        pkls_path = os.path.join(pressure_path, obj_folder)
        obj_name = get_obj_name(obj_folder)
        
        # Get sensor info from every pkl file
        all_sensors_xyz = []
        all_sensors_pressure = []
        
        for letter in "abcde":
            try:
                for pkl_name in random.sample(os.listdir(pkls_path), n):
                        pkl_path = os.path.join(pkls_path, pkl_name)
                        n_samples_dict[f"{n}_{letter}"].append(os.path.split(pkl_path)[-1])
            except:
                pass

In [ ]:
get_obj_name = lambda name : name[:len(name) - 14]
obj_names = []
n_samples_dict = defaultdict(list)

for n in n_samples:
    for obj_folder in os.listdir(pressure_path):
        pkls_path = os.path.join(pressure_path, obj_folder)
        obj_name = get_obj_name(obj_folder)
        
        # Get sensor info from every pkl file
        all_sensors_xyz = []
        all_sensors_pressure = []
        
        for letter in "abcde":
            try:
                for pkl_name in random.sample(os.listdir(pkls_path), n):
                        pkl_path = os.path.join(pkls_path, pkl_name)
                        n_samples_dict[f"{n}_{letter}"].append(os.path.split(pkl_path)[-1])
            except:
                pass